In [1]:
import xml.etree.ElementTree as ET
import datetime

In [2]:
# Parse the XML file
tree = ET.parse(r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\public\in-the-zone-2\InTheZone2_1.zwo')
# Get the root element
root = tree.getroot()

In [3]:
# Iterate over elements
for child in root:
    print(child.tag, child.attrib)
    for sub_child in child:
        print("\t", sub_child.tag, sub_child.attrib)
        # Access text content of elements if necessary
        print("\t", sub_child.text)

author {}
name {}
description {}
sportType {}
tags {}
	 tag {'name': 'ZONE2'}
	 None
workout {}
	 SteadyState {'Duration': '150', 'Power': '0.5', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.60000002', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.64999998', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.81', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '60.000004', 'Power': '0.94999999', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '120.00001', 'Power': '0.5', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '240.00002', 'Power': '0.80000001', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '120', 'Power': '0.5', 'pace': '0'}
	 
            
	 Ramp {'Duration': '360', 'PowerLow': '0.51930803', 'PowerHigh': '0.69999993', 'pace': '0'}
	 
            
	 SteadyState {'Duration': '180.00002', 'Power': '0.69999999', 'pace': '0'}
	 
            
	 Ramp

In [4]:
for element in root.find(".//workout"):
    print(element.tag)

SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
Ramp
SteadyState
Ramp
Ramp
SteadyState
Ramp
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
SteadyState
IntervalsT
SteadyState
IntervalsT
Cooldown


In [5]:
'SteadyState' in ['SteadyState','Ramp']

True

In [182]:
def parse_workout_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    workout_data = []
    intervals = []
    tagsToFind = ['SteadyState','Ramp','IntervalsT','Cooldown']
    for element in root.find(".//workout"):
        if element.tag in tagsToFind:
            intervals.append(element)
    
    for interval in intervals:
        type = interval.tag
        duration = interval.attrib.get("Duration")
        power = interval.attrib.get("Power")
        power_low = interval.attrib.get("PowerLow")
        power_high = interval.attrib.get("PowerHigh")
        onPower = interval.get('OnPower', None)
        offPower = interval.get('OffPower', None)
        onDuration = interval.get('OnDuration', None)
        offDuration = interval.get('OffDuration', None)
        repeat = interval.get('Repeat', None)

        # Convert attributes to float if they exist
        if duration is not None:
            duration = float(duration)
        if power is not None:
            power = float(power)
        if power_low is not None:
            power_low = float(power_low)
        if power_high is not None:
            power_high = float(power_high)
        if onPower is not None:
            onPower = float(onPower)
        if offPower is not None:
            offPower = float(offPower)
        if onDuration is not None:
            onDuration = float(onDuration)
        if offDuration is not None:
            offDuration = float(offDuration)
        if repeat is not None:
            repeat = int(repeat)
            
        interval_data = {
            "type": type,
            "duration": duration,
            "power": power,
            "powerLow": power_low,
            "powerHigh": power_high,
            "onPower": onPower,
            "offPower": offPower,
            "onDuration": onDuration,
            "offDuration": offDuration,
            "repeat": repeat
        }
        workout_data.append(interval_data)

    return workout_data


In [183]:
def color(power):
    if power <= 59:
        color = 'grey'
    elif power<=75:
        color = 'blue'
    elif power<=89:
        color = 'green'
    elif power<=104:
        color = 'yellow'
    elif power<=118:
        color = 'orange'   
    else:
        color = 'red'
    return color   

In [243]:
def box(time, duration, power, timeScale):
    d = datetime.timedelta(seconds=round(duration))
    d = '{:2d}:{:02d}'.format(d.seconds // 60, d.seconds % 60)
       
    path = f"""<Tooltip label = "{d}min @{round(power*100)}%">
    <path d="M {time} 0 L {time} {power*100} L {time + duration*timeScale} {power*100} L {time + duration*timeScale} {0} 0 Z" fill="{color(power*100)}" stroke="{color(power*100)}" />
    </Tooltip>"""
    return path

def trapez(time, duration, powerLeft, powerRight, timeScale):
    d = datetime.timedelta(seconds=round(duration))
    d = '{:2d}:{:02d}'.format(d.seconds // 60, d.seconds % 60)
    def pathDef(time, duration, powerLeft_, powerRight_, timeScale, color):
        path_ = f"""<Tooltip label = "{d}min ramp from {round(powerLeft*100)}% to {round(powerRight*100)}%">
        <path d="M {time} 0 L {time} {powerLeft_*100} L {time + duration*timeScale} {powerRight_*100} L {time + duration*timeScale} {0} 0 Z" fill="{color}" stroke="{color}" />
        </Tooltip>"""
        return path_
        
    if color(powerLeft*100)==color(powerRight*100):
        path = []
        path.append(pathDef(time, duration, powerLeft, powerRight, timeScale, color(powerRight*100)))
    else:
        path = []
        path.append(pathDef(time, duration, powerLeft, powerRight, timeScale, color((powerLeft+powerRight)/2)))
        
    return path
    

In [244]:
def generate_paths(workout_data):
    paths = []
    totalTime = 0
    intervals = 0
    gap = 2
    
    for interval in workout_data:
        type = interval['type']
        duration = interval.get('duration', None)
        onDuration = interval.get('onDuration', None)
        offDuration = interval.get('offDuration', None)
        repeat = interval.get('repeat', None)
                
        if type in ["SteadyState","Ramp","Cooldown"]:
            totalTime = totalTime + duration
            intervals = intervals + 1
        if type == "IntervalsT":
            totalTime = totalTime + repeat*(onDuration + offDuration)
            intervals = intervals + repeat*2
    timeScale = (1000 - intervals * gap)/totalTime
    print(totalTime)
    
    time = 0
    for interval in workout_data:
        type = interval['type']
        duration = interval.get('duration', None)
        power = interval.get('power', None)
        powerLow = interval.get('powerLow', None)
        powerHigh = interval.get('powerHigh', None)
        onPower = interval.get('onPower', None)
        offPower = interval.get('offPower', None)
        onDuration = interval.get('onDuration', None)
        offDuration = interval.get('offDuration', None)
        repeat = interval.get('repeat', None)
        
        
        if type == "SteadyState":
            path = box(time, duration, power, timeScale)
            paths.append(path)
            time = time + duration*timeScale + gap
        if type in ["Ramp","Cooldown"]:
            path = trapez(time, duration, powerLow, powerHigh, timeScale)
            paths.extend(path)
            time = time + duration*timeScale + gap
        if type == "IntervalsT":
            
            for i in range(repeat):
                path = box(time, onDuration, onPower, timeScale)
                paths.append(path)
                time = time + onDuration*timeScale + gap
                path = box(time, offDuration, offPower, timeScale)
                paths.append(path)
                time = time + offDuration*timeScale + gap
    
            
    return paths


In [245]:
workout_data

[{'type': 'SteadyState',
  'duration': 150.0,
  'power': 0.5,
  'powerLow': None,
  'powerHigh': None,
  'onPower': None,
  'offPower': None,
  'onDuration': None,
  'offDuration': None,
  'repeat': None},
 {'type': 'SteadyState',
  'duration': 60.000004,
  'power': 0.60000002,
  'powerLow': None,
  'powerHigh': None,
  'onPower': None,
  'offPower': None,
  'onDuration': None,
  'offDuration': None,
  'repeat': None},
 {'type': 'SteadyState',
  'duration': 60.000004,
  'power': 0.64999998,
  'powerLow': None,
  'powerHigh': None,
  'onPower': None,
  'offPower': None,
  'onDuration': None,
  'offDuration': None,
  'repeat': None},
 {'type': 'SteadyState',
  'duration': 60.000004,
  'power': 0.81,
  'powerLow': None,
  'powerHigh': None,
  'onPower': None,
  'offPower': None,
  'onDuration': None,
  'offDuration': None,
  'repeat': None},
 {'type': 'SteadyState',
  'duration': 60.000004,
  'power': 0.94999999,
  'powerLow': None,
  'powerHigh': None,
  'onPower': None,
  'offPower': No

In [246]:
workout_data = parse_workout_file(workout_file)
svg_paths = generate_paths(workout_data)

7200.000236000003


In [247]:
def generate_svg(paths):
    svg_start = """
    import {Box, Tooltip} from '@chakra-ui/react'
    import './css/Workout.css'

    export const WorkoutGraphSVG = () => (
    <div>
    <Box as='svg'
    viewBox='0 0 1000 100'
    width='100%'
    height={'auto'}
    transform= 'scaleY(-1)'>
    <svg>"""
    svg_end = "</svg></Box></div>);"
    svg_content = "\n".join(paths)
    return svg_start + svg_content + svg_end

In [248]:
def save_svg_to_tsx(svg_content, file_path):
    with open(file_path, 'w') as f:
        f.write(svg_content)

In [249]:
# Example usage
workout_file = r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\public\in-the-zone-2\InTheZone2_1.zwo'
workout_data = parse_workout_file(workout_file)
svg_paths = generate_paths(workout_data)
svg_content = generate_svg(svg_paths)

file_path = r'C:\Users\Chris\OneDrive\Dokumenter\React\dzr\app\in-the-zone-2\workouts\nextWorkout_.tsx'
save_svg_to_tsx(svg_content, file_path)

# Now you can use svg_paths to embed the SVG paths in your SVG container or save them to an SVG file.
# For example, you can join the svg_paths list into a single string and embed it in an SVG element in an HTML document.


7200.000236000003


In [250]:
svg_paths

['<Tooltip label = " 2:30min @50%">\n    <path d="M 0 0 L 0 50.0 L 18.54166605891205 50.0 L 18.54166605891205 0 0 Z" fill="grey" stroke="grey" />\n    </Tooltip>',
 '<Tooltip label = " 1:00min @60%">\n    <path d="M 20.54166605891205 0 L 20.54166605891205 60.000001999999995 L 27.958332976921298 60.000001999999995 L 27.958332976921298 0 0 Z" fill="blue" stroke="blue" />\n    </Tooltip>',
 '<Tooltip label = " 1:00min @65%">\n    <path d="M 29.958332976921298 0 L 29.958332976921298 64.999998 L 37.374999894930546 64.999998 L 37.374999894930546 0 0 Z" fill="blue" stroke="blue" />\n    </Tooltip>',
 '<Tooltip label = " 1:00min @81%">\n    <path d="M 39.374999894930546 0 L 39.374999894930546 81.0 L 46.791666812939795 81.0 L 46.791666812939795 0 0 Z" fill="green" stroke="green" />\n    </Tooltip>',
 '<Tooltip label = " 1:00min @95%">\n    <path d="M 48.791666812939795 0 L 48.791666812939795 94.999999 L 56.20833373094904 94.999999 L 56.20833373094904 0 0 Z" fill="yellow" stroke="yellow" />\n   